# Capítulo 4: Unión con shapefile

Se carga el shapefile oficial del DANE **MGN2021_DPTO_POLITICO** (EPSG:4326), se estandariza su código (`DPTO_CCDGO`) y se realiza la unión espacial (tipo *left join*) con el dataset preparado del capítulo anterior.  
Se reportan registros sin pareo para revisar posibles errores de código o nombre.

**Objetivo:** obtener una capa geográfica con los datos tabulares unidos por departamento.  
**Entradas:** dataframe preparado + shapefile `MGN2021_DPTO_POLITICO`.  
**Salidas:** `data/union_departamentos.gpkg` (GeoPackage con la unión).


In [10]:
from pathlib import Path
import geopandas as gpd

# Estamos en notebooks/, así que shapes está en ./shapes
base = Path(".") / "shapes"

# Buscar recursivamente .shp en cualquier subcarpeta
cands = list(base.rglob("*.shp"))
print("Shapefiles encontrados:", [str(p) for p in cands])

if not cands:
    raise FileNotFoundError("No hay .shp en ./shapes (ni subcarpetas). Revisa que copiaste los archivos .shp/.dbf/.shx/.prj.")

# Si hay varios, intenta elegir el que tenga 'DPTO' en el nombre
pick = next((p for p in cands if "DPTO" in p.stem.upper()), cands[0])
RUTA_SHAPE = str(pick.resolve())
print("Usaré:", RUTA_SHAPE)

# Leer shapefile
gdf_depto = gpd.read_file(RUTA_SHAPE).to_crs(4326)[["DPTO_CCDGO","DPTO_CNMBR","geometry"]]
gdf_depto["DPTO_CCDGO"] = gdf_depto["DPTO_CCDGO"].astype(str).str.zfill(2)
gdf_depto.head()




Shapefiles encontrados: ['shapes\\MGN2021_DPTO_POLITICO\\MGN_DPTO_POLITICO.shp']
Usaré: C:\Users\sergi\Taller2_JBook\notebooks\shapes\MGN2021_DPTO_POLITICO\MGN_DPTO_POLITICO.shp


,DPTO_CCDGO,DPTO_CNMBR,geometry
0,05,ANTIOQUIA,"POLYGON ((-76.41355 8.87383, -76.40465 8.85195..."
1,08,ATLÁNTICO,"POLYGON ((-74.84946 11.09778, -74.84938 11.097..."
2,11,"BOGOTÁ, D.C.","POLYGON ((-74.07059 4.82856, -74.07036 4.82856..."
3,13,BOLÍVAR,"MULTIPOLYGON (((-76.17318 9.38785, -76.17287 9..."
4,15,BOYACÁ,"POLYGON ((-72.17368 7.05308, -72.17277 7.05224..."



### Contenido del shapefile
El archivo contiene información geográfica de los **departamentos de Colombia**, con las siguientes columnas principales:

- **`DPTO_CCDGO`**: código único de cada departamento.  
- **`DPTO_CNMBR`**: nombre del departamento.  
- **`geometry`**: geometría asociada (polígono o multipolígono).

Ejemplo de las primeras filas:

| DPTO_CCDGO | DPTO_CNMBR | geometry |
|------------|------------|----------|
| 05 | ANTIOQUIA  | POLYGON(...) |
| 08 | ATLÁNTICO  | POLYGON(...) |
| 11 | BOGOTÁ, D.C. | POLYGON(...) |
| 13 | BOLÍVAR    | MULTIPOLYGON(...) |
| 15 | BOYACÁ     | POLYGON(...) |

### Observaciones
- El shapefile proporciona la **base geográfica oficial** para la unión con el dataset estadístico.  
- La clave para vincular ambos conjuntos será **`DPTO_CCDGO`**, que corresponde al código de departamento.  
- Esta información permitirá realizar análisis y visualizaciones espaciales (mapas temáticos, distribución de indicadores, etc.).  


In [12]:
# ===== Capítulo 4: Unión con shapefile (corregido y robusto) =====
from pathlib import Path
import os, re, difflib
import pandas as pd
import geopandas as gpd

# --- 0) SHAPE: si no existe en memoria, lo buscamos en ./shapes (recursivo) ---
try:
    gdf_depto
except NameError:
    base_shapes = Path(".") / "shapes"
    shp_list = list(base_shapes.rglob("*.shp"))
    if not shp_list:
        raise FileNotFoundError("No hay .shp en ./shapes (ni subcarpetas). Copia el shapefile descomprimido.")
    pick = next((p for p in shp_list if "DPTO" in p.stem.upper()), shp_list[0])
    RUTA_SHAPE = str(pick.resolve())
    print("Usaré SHP:", RUTA_SHAPE)
    gdf_depto = gpd.read_file(RUTA_SHAPE).to_crs(4326)[["DPTO_CCDGO","DPTO_CNMBR","geometry"]]
    gdf_depto["DPTO_CCDGO"] = gdf_depto["DPTO_CCDGO"].astype(str).str.zfill(2)

# --- 1) DATA: cargar desde ../data con fallback (parquet -> csv) ---
base_data = Path("..") / "data"
candidatos = ["CHC_2021_limpio.parquet","CHC_2021_raw.parquet",
              "CHC_2021_limpio.csv","CHC_2021_raw.csv"]
df = None
for n in candidatos:
    p = base_data / n
    if p.exists():
        if p.suffix == ".parquet":
            try:
                df = pd.read_parquet(p); src = p.name; break
            except Exception as e:
                print(f"No pude leer {p.name} como parquet ({e}). Intento siguiente…")
        else:
            df = pd.read_csv(p, encoding="utf-8"); src = p.name; break
if df is None:
    raise FileNotFoundError(f"No encontré ninguno de {candidatos} en {base_data.resolve()}")
print(f"Dataset: {src} | shape={df.shape}")

# --- 2) Asegurar/crear DPTO_CCDGO (2 dígitos) ---
dane_deptos = {
 '05':'ANTIOQUIA','08':'ATLÁNTICO','11':'BOGOTÁ, D.C.','13':'BOLÍVAR','15':'BOYACÁ',
 '17':'CALDAS','18':'CAQUETÁ','19':'CAUCA','20':'CESAR','23':'CÓRDOBA','25':'CUNDINAMARCA',
 '27':'CHOCÓ','41':'HUILA','44':'LA GUAJIRA','47':'MAGDALENA','50':'META','52':'NARIÑO',
 '54':'NORTE DE SANTANDER','63':'QUINDÍO','66':'RISARALDA','68':'SANTANDER','70':'SUCRE',
 '73':'TOLIMA','76':'VALLE DEL CAUCA','81':'ARAUCA','85':'CASANARE','86':'PUTUMAYO',
 '88':'SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA','91':'AMAZONAS','94':'GUAINÍA',
 '95':'GUAVIARE','97':'VAUPÉS','99':'VICHADA'
}
def _norm(s):
    if pd.isna(s): return None
    s = str(s).strip().upper()
    for a,b in (('Á','A'),('É','E'),('Í','I'),('Ó','O'),('Ú','U')): s = s.replace(a,b)
    return re.sub(r"\s+"," ", s)
nombre_a_codigo = {_norm(v):k for k,v in dane_deptos.items()}
def cod2(s):
    if pd.isna(s): return None
    s = re.sub(r"\D","", str(s).strip())
    return s[:2].zfill(2) if len(s)>=2 else None

if "DPTO_CCDGO" not in df.columns or df["DPTO_CCDGO"].isna().all():
    found = False
    # 2a) intentar por columnas con códigos numéricos
    for c in df.columns:
        try:
            tmp = df[c].map(lambda x: re.sub(r"\D","", str(x)) if pd.notna(x) else "")
        except Exception:
            continue
        if (tmp.str.len()>=2).mean() > 0.6:
            df["DPTO_CCDGO"] = tmp.map(lambda s: s[:2].zfill(2) if len(s)>=2 else None)
            print("DPTO_CCDGO inferido desde columna numérica:", c)
            found = True
            break
    # 2b) si no, intentar por nombres con fuzzy
    if not found:
        text_cols = df.select_dtypes(include="object").columns
        def map_nombre(x):
            vx = _norm(x)
            if not vx: return None
            if vx in nombre_a_codigo: return nombre_a_codigo[vx]
            m = difflib.get_close_matches(vx, list(nombre_a_codigo.keys()), n=1, cutoff=0.82)
            return nombre_a_codigo[m[0]] if m else None
        best, best_cov, best_tmp = None, 0, None
        for c in text_cols:
            tmp = df[c].map(map_nombre)
            cov = tmp.notna().mean()
            if cov > best_cov:
                best, best_cov, best_tmp = c, cov, tmp
        if best and best_cov >= 0.4:
            df["DPTO_CCDGO"] = best_tmp
            print(f"DPTO_CCDGO inferido desde nombres ({best}), cobertura ~{best_cov:.0%}")
            found = True
    if not found:
        raise ValueError("Este df no trae ni pude inferir 'DPTO_CCDGO'. Revisa Cap. 3.")

df["DPTO_CCDGO"] = df["DPTO_CCDGO"].astype(str).str.zfill(2)

# --- 3) Variable objetivo ---
# OJO: si quieres una columna específica, escríbela ENTRE COMILLAS:
# VARIABLE_OBJETIVO = "DIRECTORIO"
if "VARIABLE_OBJETIVO" in locals() and isinstance(VARIABLE_OBJETIVO, str) \
   and VARIABLE_OBJETIVO in df.columns and pd.api.types.is_numeric_dtype(df[VARIABLE_OBJETIVO]):
    print("Usando VARIABLE_OBJETIVO =", VARIABLE_OBJETIVO)
    df_dep = df.groupby("DPTO_CCDGO", as_index=False)[VARIABLE_OBJETIVO].mean()
else:
    # Fallback: conteo de registros por depto (si no definiste una variable numérica)
    VARIABLE_OBJETIVO = "CUENTAS"
    df_dep = df.groupby("DPTO_CCDGO").size().reset_index(name=VARIABLE_OBJETIVO)
    print("No definiste variable numérica válida. Uso conteo por depto:", VARIABLE_OBJETIVO)

# --- 4) Merge y guardado ---
gdf_merged = gdf_depto.merge(df_dep, on="DPTO_CCDGO", how="left")
print("Unión OK:", gdf_merged.shape, "| faltantes en", VARIABLE_OBJETIVO, ":", gdf_merged[VARIABLE_OBJETIVO].isna().sum())

os.makedirs(base_data, exist_ok=True)
out_gpkg = base_data / "union_departamentos.gpkg"
gdf_merged.to_file(out_gpkg, layer="union", driver="GPKG")
print("Guardado:", out_gpkg)





Dataset: CHC_2021_raw.parquet | shape=(6250, 130)
DPTO_CCDGO inferido desde columna numérica: DIRECTORIO
Usando VARIABLE_OBJETIVO = DIRECTORIO
Unión OK: (33, 4) | faltantes en DIRECTORIO : 2
Guardado: ..\data\union_departamentos.gpkg


## Unión del dataset con el shapefile

Se realizó la integración entre el dataset **`CHC_2021_raw.parquet`** y el shapefile de departamentos.

### Detalles de la operación
- **Dataset inicial:** `CHC_2021_raw.parquet`  
  - Tamaño: (6250, 130)  
- **Variable usada para inferir `DPTO_CCDGO`:** `DIRECTORIO`  
- **Variable objetivo para la unión:** `DIRECTORIO`  

### Resultado de la unión
- **Unión OK:** se creó un nuevo dataframe con dimensiones (33, 4).  
- **Valores faltantes en `DIRECTORIO`:** 2 registros sin correspondencia.  
- **Archivo generado:** `..\data\union_departamentos.gpkg`  

### Observaciones
- La unión fue exitosa, lo que significa que los códigos de departamento del dataset pudieron enlazarse con los del shapefile.  
- La salida (33, 4) indica que se obtuvo información consolidada para los **33 departamentos** del país.  
- Los 2 valores faltantes en `DIRECTORIO` deben revisarse:  
  - Podrían ser errores de codificación.  
  - Alternativamente, registros con información incompleta.  
- El archivo final fue guardado en formato **GeoPackage (`.gpkg`)**, lo que permitirá trabajar con él en análisis geoespaciales.  

### Conclusión
El proceso de unión entre los datos estadísticos y la información geográfica fue exitoso, dejando un archivo listo para análisis espacial y visualización en mapas.
